In [2]:
from influxdb import InfluxDBClient
from ruamel.yaml import YAML
import pandas as pd
import datetime as dt
from streamlit_app import Trace, JsonDec, JsonEnc
import json, os
import pandas as pd
import datetime as dt
import inspect
from dataclasses import dataclass, field, asdict
from typing import Callable
from ruamel.yaml import YAML

In [3]:
with open("secrets.yaml", "r") as file:
    secrets = YAML().load(file)

client = InfluxDBClient(host=secrets["influx"]["host"], 
                        port=secrets["influx"]["port"], 
                        username=secrets["influx"]["username"], 
                        password=secrets["influx"]["password"])

client.get_list_database()

[{'name': '_internal'},
 {'name': 'influx_db_1'},
 {'name': 'influx_db_2'},
 {'name': 'influx_db_3'},
 {'name': 'influx_db_4'}]

In [19]:
client.switch_database('influx_db_3')
entity_id = "benni_battery"
selected_unit = "%"
qstr = f"""SELECT value, mean_value FROM "{selected_unit}" WHERE entity_id = '{entity_id}'"""
df = pd.DataFrame.from_records(client.query(qstr).get_points())
df = df.dropna(axis=1).set_index("time")
print(f"{df.shape=}")
series = df[df.columns[0]]
series

df.shape=(74, 1)


time
2024-03-16T10:39:53.228986Z    100.0
2024-03-16T11:25:19.423893Z    100.0
2024-03-16T11:57:42.335223Z    100.0
2024-03-16T13:35:36.641544Z    100.0
2024-03-16T14:51:17.549563Z    100.0
                               ...  
2024-04-22T15:01:22.988830Z    100.0
2024-04-24T05:09:08.109779Z    100.0
2024-04-24T18:55:00.344332Z    100.0
2024-04-25T10:11:29.028782Z    100.0
2024-05-01T09:48:33.094654Z    100.0
Name: value, Length: 74, dtype: float64

time
2024-03-16T10:00:00Z    100.0
2024-03-16T11:00:00Z    100.0
2024-03-16T13:00:00Z    100.0
2024-03-16T14:00:00Z    100.0
2024-03-16T20:00:00Z    100.0
                        ...  
2024-04-22T15:00:00Z    100.0
2024-04-24T05:00:00Z    100.0
2024-04-24T18:00:00Z    100.0
2024-04-25T10:00:00Z    100.0
2024-05-01T09:00:00Z    100.0
Name: mean_value, Length: 62, dtype: float64